# Experiment: Automatically find grid boundaries

In [3]:
mode="hyperspectral"

In [4]:
cap=500
index=130

In [5]:
# Path to the directory
data="/media/gmobot/Elements_24/Elements_24/grid_testing/newgrid_103023/"

In [8]:
import os
from spectral import *
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


def plot_on_axes(ax, data, style, title, legend_label=None, x_data=None):
    if x_data is None:
        x_data = range(len(data))
    ax.plot(x_data, data, style, label=legend_label)
    ax.set_title(title)
    if legend_label:
        ax.legend()

def plot_derivatives(row_avg, row_avg_smooth,
                     col_avg, col_avg_smooth,
                     row_derivative1, col_derivative1,
                     row_derivative2, col_derivative2,
                     window_size, mode):
    
    if mode == "hyperspectral":
        mode = "Hyperspectral"

    # Plotting
    fig, axs = plt.subplots(3, 2, figsize=(18, 12))
    
    # Average values
    plot_on_axes(axs[0, 0], row_avg, 'o-', mode + ' Row Averages', 'Original')
    plot_on_axes(axs[0, 0], row_avg_smooth, 'x-', mode + ' Row Averages', 'Smoothed', range(window_size-1, len(row_avg)))

    plot_on_axes(axs[0, 1], col_avg, 'o-', mode + ' Column Averages', 'Original')
    plot_on_axes(axs[0, 1], col_avg_smooth, 'x-', mode + ' Column Averages', 'Smoothed', range(window_size-1, len(col_avg)))

    # First derivative
    plot_on_axes(axs[1, 0], row_derivative1, 'x-', mode + ' First Derivative of Smoothed Row Averages')
    plot_on_axes(axs[1, 1], col_derivative1, 'x-', mode + ' First Derivative of Smoothed Column Averages')

    # Second derivative
    plot_on_axes(axs[2, 0], row_derivative2, 's-', mode + ' Second Derivative of Smoothed Row Averages')
    plot_on_axes(axs[2, 1], col_derivative2, 's-', mode + ' Second Derivative of Smoothed Column Averages')

    plt.tight_layout()
    plt.show()

def plot_heatmap(mode, CLS_matrix, y1, y2, x1, x2):
    if mode == "hyperspectral":
        mode = "Hyperspectral"
    
    # Plotting the heatmap
    fig, axs = plt.subplots(1, 1)
    
    im = axs.imshow(CLS_matrix, cmap='viridis', aspect='auto')
    axs.set_title(f'{mode} signal intensity at each position')
    axs.set_xlabel('X position')
    axs.set_ylabel('Y position')

    # Adding red lines
    axs.axhline(y=y1, color='r', linestyle='--')
    axs.axhline(y=y2, color='r', linestyle='--')
    axs.axvline(x=x1, color='r', linestyle='--')
    axs.axvline(x=x2, color='r', linestyle='--')

    # Adding a colorbar for intensity
    cbar = plt.colorbar(im, ax=axs)
    cbar.set_label('Intensity')
    
    plt.show()

# List all files in the directory
all_files = os.listdir(data)

if mode == "hyperspectral" :

    selected_file = [f for f in all_files if "Broadband" in f and "hroma" in f and "hdr" in f]

    # Assert that the length of the filtered files is 1
    assert len(selected_file) == 1, f"Expected 1 file, found {len(selected_file)}"

    # Print or return the filtered file
    print("Selected file:", selected_file[0])

    hdr_path=os.path.join(data, selected_file[0])

    image1 = envi.open(hdr_path)

    print("Extracting channel at index " + str(index))

    CLS_matrix = np.squeeze(image1[:,:,index],
                            axis=2)

    CLS_matrix = np.interp(CLS_matrix,
                           (0,
                            cap),
                           (0,
                            255)).astype(int)
    
if mode == "RGB":

    # Load the image
    selected_file = [f for f in all_files if "hroma" in f and "jpg" in f]
    jpg_path=os.path.join(data, selected_file[0])
    image = Image.open(jpg_path)

    # Convert to grayscale
    image_gray = image.convert("L")

    # Create CLS_matrix from the grayscale image
    CLS_matrix = np.array(image_gray)

# Calculate averages for each row and each column
row_avg = np.mean(CLS_matrix, axis=1)
col_avg = np.mean(CLS_matrix, axis=0)

# Apply a moving average with a window size of 3
window_size = 4
row_avg_smooth = moving_average(row_avg, window_size)
col_avg_smooth = moving_average(col_avg, window_size)

# Compute the first and second derivatives of the smoothed data
row_derivative1 = np.diff(row_avg_smooth)
row_derivative2 = np.diff(row_derivative1)

col_derivative1 = np.diff(col_avg_smooth)
col_derivative2 = np.diff(col_derivative1)

y1 = np.argmax(row_derivative1)
y2 = np.argmin(row_derivative1)
x1 = np.argmax(col_derivative1)
x2 = np.argmin(col_derivative1)
    
plot_derivatives(row_avg, row_avg_smooth, col_avg, col_avg_smooth,
                 row_derivative1, col_derivative1, row_derivative2, col_derivative2,
                 window_size, mode)

plot_heatmap(mode, CLS_matrix, y1, y2, x1, x2)